# K1 — 0.0 — OTIMIZAÇÃO - BASE DE DADOS EXPANDIDA
**Objetivo:** Criar uma versão Kaggle Otimizada consolidando informações dispersas em várias tabelas, com foco em colunas que são comparáveis às da Lighthouse (para consistência) e em colunas adicionais úteis para análises/modelos (para enriquecer).

**Saída principal:** Dataset kaggle otimizado  `data/processed/kaggle_otimizado.csv`

# K1 — 1.0 — IMPORTS

In [1]:
# =====================================================
# Imports
# =====================================================

# Manipulação e análise de dados
import math
import numpy as np
import pandas as pd
import re
from pathlib import Path
import json, ast

# Visualização de dados
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from tabulate import tabulate

# Sistema e paths
import os
from pathlib import Path

# Utilidades para notebooks
from IPython.display import display
from IPython.display import Image
from IPython.core.display import HTML

# Manipulação de datas
import datetime

# Supress warnings
import warnings

In [2]:
warnings.filterwarnings('ignore')

# K1 — 2.0 — FUNÇÕES AUXILIARES

In [3]:
# ==== Caminhos & Display ====
class PATHS:
    """
    Caminhos padrão do projeto quando o notebook roda dentro de 'notebooks/'.

    Regra:
    - Se o cwd termina com 'notebooks', a raiz é o pai (../)
    - Caso contrário, assume que já estamos na raiz (útil se abrir o Jupyter na raiz)
    """
    _CWD = Path.cwd()
    ROOT = _CWD.parent if _CWD.name == "notebooks" else _CWD
    RAW  = ROOT / "data" / "raw"
    PROC = ROOT / "data" / "processed"
    INTER = ROOT / "data" / "intermediary"
    REP  = ROOT / "reports"
    FIG  = REP / "figures"

def ensure_dirs():
    """Garante que as pastas principais existam."""
    for p in [PATHS.RAW, PATHS.INTER, PATHS.PROC, PATHS.REP, PATHS.FIG]:
        p.mkdir(parents=True, exist_ok=True)

def set_display(max_cols: int = 100, decimals: int = 2):
    """Ajusta visualização padrão no Pandas/Seaborn para leitura eficiente."""
    pd.options.display.max_columns = max_cols
    fmt = "{:." + str(decimals) + "f}"
    pd.options.display.float_format = fmt.format
    sns.set(style="whitegrid", palette="muted", font_scale=1.1)

# ==== Parsing & Normalização ====
def normalize_title(s):
    """Normaliza título: minúsculas, sem pontuação, sem espaços extras."""
    if pd.isna(s):
        return np.nan
    s = s.lower()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^\w\s]", "", s)
    return s.strip()

def parse_year(x):
    """Extrai o primeiro ano encontrado (ex.: '1994' ou '1994(I)')."""
    if pd.isna(x):
        return pd.NA
    m = re.search(r"\d{4}", str(x))
    return int(m.group()) if m else pd.NA

def to_numeric_safe(series):
    """Converte para numérico com 'coerce' (incompatíveis viram NaN)."""
    return pd.to_numeric(series, errors="coerce")

# ==== Deduplicação ====
def dedup_by_key(df, keys, score_col="vote_count"):
    """
    Se houver múltiplas linhas para a mesma chave, mantém a de maior `score_col`.
    Ex.: ao agrupar por (title_norm, Year), fica a linha com maior número de votos.
    """
    tmp = df.copy()
    tmp[score_col] = to_numeric_safe(tmp[score_col]) if score_col in tmp.columns else 0
    tmp = tmp.sort_values(by=[score_col], ascending=False)
    return tmp.drop_duplicates(subset=keys, keep="first")

In [4]:
# Chamo as de caminhos e display
ensure_dirs()
set_display()

# K1 — 3.0 — CARREGAMENTO DOS DADOS

In [5]:
# Realizo a leitura dos dados através da classe
kc = pd.read_csv(PATHS.RAW / "kaggle_credits.csv", low_memory=False)
kk = pd.read_csv(PATHS.RAW / "kaggle_keywords.csv", low_memory=False)          
kl = pd.read_csv(PATHS.RAW / "kaggle_links.csv", low_memory=False)  
kmm = pd.read_csv(PATHS.RAW / "kaggle_movies_metadata.csv", low_memory=False)  
kr = pd.read_csv(PATHS.RAW / "kaggle_ratings.csv", low_memory=False)  

# K1 — 4.0 — MERGE DAS TABELAS

## K1 — 4.1 — Incorporando Diretor e Atores Principais

In [6]:
# Faço uma cópia da tabela de credits
credits = kc.copy()

# Converto colunas de string JSON para listas/dicionários Python
for col in ["cast", "crew"]:
    credits[col] = credits[col].map(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

# Extraio o diretor
def get_director(crew_list):
    for person in crew_list:
        if person.get("job") == "Director":
            return person.get("name")
    return np.nan

credits["director"] = credits["crew"].map(get_director)

# Extraio até 4 atores principais
def get_actors(cast_list, top_n=4):
    names = [person.get("name") for person in cast_list[:top_n]]
    while len(names) < top_n:
        names.append(np.nan)  # preenche se tiver menos de 4
    return names

actors_expanded = credits["cast"].map(lambda x: get_actors(x, top_n=4))
actors_df = pd.DataFrame(actors_expanded.tolist(), columns=["star1","star2","star3","star4"])

# Junto no DataFrame credits
credits = pd.concat([credits[["id","director"]], actors_df], axis=1)

display(credits.head())

,id,director,star1,star2,star3,star4
0,862,John Lasseter,Tom Hanks,Tim Allen,Don Rickles,Jim Varney
1,8844,Joe Johnston,Robin Williams,Jonathan Hyde,Kirsten Dunst,Bradley Pierce
2,15602,Howard Deutch,Walter Matthau,Jack Lemmon,Ann-Margret,Sophia Loren
3,31357,Forest Whitaker,Whitney Houston,Angela Bassett,Loretta Devine,Lela Rochon
4,11862,Charles Shyer,Steve Martin,Diane Keaton,Martin Short,Kimberly Williams-Paisley


In [7]:
# Padronizo a chave de merge entre as tabelas para evitar erros
kmm["id"] = pd.to_numeric(kmm["id"], errors="coerce").astype("Int64")
credits["id"] = pd.to_numeric(credits["id"], errors="coerce").astype("Int64")

In [8]:
print("kmm id nulos:", kmm["id"].isna().sum())
print("credits id nulos:", credits["id"].isna().sum())

kmm id nulos: 3
credits id nulos: 0


In [9]:
# Junto as tabelas
kmm_credits = kmm.merge(credits, on="id", how="left")

In [10]:
# Realizo checagem rápida
print(kmm_credits.shape)
print(kmm_credits[["id","title","director","star1","star2","star3","star4"]].head(3))
print("Cobertura com diretor:", kmm_credits["director"].notna().mean())

(45542, 29)
      id             title       director           star1          star2  \
0    862         Toy Story  John Lasseter       Tom Hanks      Tim Allen   
1   8844           Jumanji   Joe Johnston  Robin Williams  Jonathan Hyde   
2  15602  Grumpier Old Men  Howard Deutch  Walter Matthau    Jack Lemmon   

           star3           star4  
0    Don Rickles      Jim Varney  
1  Kirsten Dunst  Bradley Pierce  
2    Ann-Margret    Sophia Loren  
Cobertura com diretor: 0.9804356418251284


## K1 — 4.2 — Incorporando Ratings dos Filmes

In [11]:
# Preparo a tabela links: alinhar imdbId com formato 'ttxxxxx'
links = kl.copy()
links["imdb_id"] = "tt" + links["imdbId"].astype(str).str.zfill(7)

print("Links shape:", links.shape)
display(links.head(3))

# Agrego os ratings por filme (vários usuários deram notas para o mesmo filme)
ratings_agg = (
    kr.groupby("movieId")["rating"]
       .agg(mean_rating="mean", num_ratings="count")
       .reset_index()
)

print("Ratings agregados:", ratings_agg.shape)
display(ratings_agg.head(3))

# Junto os links com ratings
links_ratings = links.merge(ratings_agg, on="movieId", how="left")

print("Links + Ratings:", links_ratings.shape)
display(links_ratings.head(3))

# Junto na base principal (kmm_credits) atrvés do links_ratings
kmm_full = kmm_credits.merge(
    links_ratings[["imdb_id","mean_rating","num_ratings"]],
    on="imdb_id",
    how="left"
)

print("Base final (com ratings):", kmm_full.shape)

Links shape: (45843, 4)


,movieId,imdbId,tmdbId,imdb_id
0,1,114709,862.00,tt0114709
1,2,113497,8844.00,tt0113497
2,3,113228,15602.00,tt0113228


Ratings agregados: (45115, 3)


,movieId,mean_rating,num_ratings
0,1,3.89,66008
1,2,3.24,26060
2,3,3.18,15497


Links + Ratings: (45843, 6)


,movieId,imdbId,tmdbId,imdb_id,mean_rating,num_ratings
0,1,114709,862.00,tt0114709,3.89,66008.00
1,2,113497,8844.00,tt0113497,3.24,26060.00
2,3,113228,15602.00,tt0113228,3.18,15497.00


Base final (com ratings): (45542, 31)


In [12]:
# Checo a cobertura final
cobertura_rating = kmm_full["mean_rating"].notna().mean()
print(f"Cobertura de filmes com ratings: {cobertura_rating:.1%}")

display(kmm_full.sample(5, random_state=42))

Cobertura de filmes com ratings: 98.2%


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,director,star1,star2,star3,star4,mean_rating,num_ratings
40658,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,382892,tt5096628,fr,Théo et Hugo dans le même bateau,"Théo and Hugo meet in a sex club, recognize ea...",1.730661,/iwjWsXxJZhdDs1fx3ohaiLsY1a5.jpg,"[{'name': 'Ecce Films', 'id': 7128}, {'name': ...","[{'iso_3166_1': 'FR', 'name': 'France'}]",2016-04-27,0.00,97.00,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Paris 05:59: Théo & Hugo,False,7.50,17.00,Olivier Ducastel,Geoffrey Couet,François Nambot,Georges Daaboul,Elodie Adler,4.08,6.00
5874,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,15395,tt0282698,en,Love Liza,Following the unexplained suicide of his wife ...,2.362585,/oSvvgD9j4VgYdeeZrsMgV0kfAJT.jpg,[],[],2002-12-30,0.00,90.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Love Liza,False,6.50,25.00,Todd Louiso,Philip Seymour Hoffman,Jack Kehler,Sarah Koskoff,Kathy Bates,3.37,356.00
17023,False,NaN,12300000,"[{'id': 35, 'name': 'Comedy'}]",NaN,18826,tt0814331,en,Spring Breakdown,Three women in their thirties head to a popula...,2.355305,/rP6wvCc6zH6GJvKsSmdFInAI1Qh.jpg,"[{'name': 'Code Entertainment', 'id': 14589}]","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-05-22,0.00,84.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Spring Breakdown,False,5.10,18.00,Ryan Shiraki,Amy Poehler,Parker Posey,Rachel Dratch,Amber Tamblyn,2.29,21.00
28120,False,NaN,0,"[{'id': 27, 'name': 'Horror'}]",NaN,32322,tt0074430,en,"Dr. Black, Mr. Hyde",A doctor (Bernie Casey) in a Watts clinic take...,0.000578,/aVSna1LHOIIAAI0f8vAmcS9b6nO.jpg,"[{'name': 'Dimension Pictures', 'id': 4997}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1976-01-01,0.00,85.00,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Fear of the Year is Here!,"Dr. Black, Mr. Hyde",False,6.30,3.00,William Crain,Bernie Casey,Rosalind Cash,Marie O'Henry,Ji-Tu Cumbuka,2.00,2.00
10428,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,10703,tt0429078,zh,七劍,Seven warriors come together to protect a vill...,4.360738,/fbMVirC0QIlzxMJabtH8OqWFCtd.jpg,"[{'name': 'Boram Entertainment Inc.', 'id': 29...","[{'iso_3166_1': 'KR', 'name': 'South Korea'}, ...",2005-07-25,0.00,153.00,"[{'iso_639_1': 'cn', 'name': '广州话 / 廣州話'}, {'i...",Released,NaN,Seven Swords,False,6.00,46.00,Tsui Hark,Leon Lai,Charlie Yeung,Lu Yi,Liu Chia-Liang,3.20,53.00


A base original do Kaggle é relacional, distribuída em várias tabelas. Para agilizar a análise e a modelagem, consolidei em uma única tabela com atributos essenciais: financeiros, notas, votos, diretor, atores, gênero e descrições.

Os joins foram feitos respeitando as chaves de ligação: id (tmdbId) para credits/keywords e links como ponte entre ratings e metadata.

As colunas não relevantes ou redundantes (ex.: poster_path, links técnicos, flags de adulto/vídeo) foram descartadas para reduzir ruído e deixar o dataset mais enxuto.

Essa consolidação garante que temos um dataset único e consistente para responder às perguntas de negócio e treinar modelos.

# K1 — 5.0 — SALVANDO A BASE DE DADOS FINAL

In [13]:
# Salvo o dataset completo do Kaggle
out_full  = PATHS.INTER / "kaggle_movies_metadata_full.csv"

kmm_full.to_csv(out_full, index=False)

print("Arquivos salvos:")
print("Full:", out_full.resolve())

Arquivos salvos:
Full: /home/emersds/repos_projetos/project_lighthouse/data/intermediary/kaggle_movies_metadata_full.csv


# K1 — 6.0 — OTIMIZAÇÃO DA BASE FINAL (PRÉ SELEÇÃO E LIMPEZA)

In [14]:
kmm_opt = kmm_full.copy()

## K1 — 6.1 — Primeira seleção de variáveis (features)

In [15]:
cols_drop = [
    "adult",
    "belongs_to_collection",
    "homepage",
    "poster_path",
    "production_companies",
    "spoken_languages",
    "status",
    "tagline",
    "video",
    "title",
    "id"
]

# Dropo apenas as colunas acima e que existir para evitar erro
kmm_opt = kmm_opt.drop(columns=[c for c in cols_drop if c in kmm_opt.columns])

In [16]:
# Checando
print("Antes:", kmm_full.shape)
print("Depois:", kmm_opt.shape)
print("Colunas atuais:", list(kmm_opt.columns))

Antes: (45542, 31)
Depois: (45542, 20)
Colunas atuais: ['budget', 'genres', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'production_countries', 'release_date', 'revenue', 'runtime', 'vote_average', 'vote_count', 'director', 'star1', 'star2', 'star3', 'star4', 'mean_rating', 'num_ratings']


## K1 — 6.2 — Normalização de títulos, datas e numéricos

In [17]:
kmm_opt.dtypes

budget                   object
genres                   object
imdb_id                  object
original_language        object
original_title           object
overview                 object
popularity               object
production_countries     object
release_date             object
revenue                 float64
runtime                 float64
vote_average            float64
vote_count              float64
director                 object
star1                    object
star2                    object
star3                    object
star4                    object
mean_rating             float64
num_ratings             float64
dtype: object

In [18]:
# Normalizo os títulos 
kmm_opt["original_title_norm"] = kmm_opt["original_title"].map(normalize_title)

In [19]:
# Padronizo os tipos das colunas (numericas)
num_cols = ["budget","popularity","vote_count","vote_average","mean_rating","num_ratings"]
for c in num_cols:
    if c in kmm_opt.columns:
        kmm_opt[c] = to_numeric_safe(kmm_opt[c])

In [20]:
# Crio o ano
kmm_opt["release_date"] = pd.to_datetime(kmm_opt["release_date"], errors="coerce")
kmm_opt["year"] = kmm_opt["release_date"].map(parse_year).astype("Int64")

In [21]:
# Normalizo as colunas categoricas
cat_cols = ["original_language","production_countries","director","star1","star2","star3","star4"]
for c in cat_cols:
    if c in kmm_opt.columns:
        kmm_opt[c] = (
            kmm_opt[c]
              .astype(str)
              .str.strip()
              .replace({"nan": np.nan})
        )

In [22]:
kmm_opt.dtypes

budget                         float64
genres                          object
imdb_id                         object
original_language               object
original_title                  object
overview                        object
popularity                     float64
production_countries            object
release_date            datetime64[ns]
revenue                        float64
runtime                        float64
vote_average                   float64
vote_count                     float64
director                        object
star1                           object
star2                           object
star3                           object
star4                           object
mean_rating                    float64
num_ratings                    float64
original_title_norm             object
year                             Int64
dtype: object

In [23]:
kmm_opt = kmm_opt.drop(columns=["original_title"])

In [24]:
# Conto quantos registros estão duplicados por (original_title_norm, year)
dup_mask = kmm_opt.duplicated(subset=["original_title_norm", "year"], keep=False)
print("Duplicatas antes:", dup_mask.sum())
kmm_opt.shape

Duplicatas antes: 198


(45542, 21)

In [25]:
# Se houver múltiplas linhas para a mesma chave, mantenho a de maior votos
kmm_opt = dedup_by_key(kmm_opt, keys=["original_title_norm", "year"], score_col="vote_count")

In [26]:
dup_mask_after = kmm_opt.duplicated(subset=["original_title_norm", "year"], keep=False)
print("Duplicatas depois:", dup_mask_after.sum())
kmm_opt.shape

Duplicatas depois: 0


(45411, 21)

## K1 — 6.3 — Parsing de colunas JSON

In [27]:
def _to_list_of_dicts(val):
    """Converte a célula (string JSON, lista de dicts, vazio) para uma lista de dicts segura."""
    # 1) valores vazios/NaN
    if val is None or (isinstance(val, float) and pd.isna(val)):
        return []
    # 2) já é lista
    if isinstance(val, list):
        return [x for x in val if isinstance(x, dict)]
    # 3) string -> tentar literal_eval, depois json.loads
    if isinstance(val, str):
        s = val.strip()
        if not s or s.lower() in {"nan", "none", "null"}:
            return []
        for loader in (ast.literal_eval, json.loads):
            try:
                obj = loader(s)
                break
            except Exception:
                obj = None
        if isinstance(obj, list):
            return [x for x in obj if isinstance(x, dict)]
        else:
            return []
    # 4) qualquer outro tipo
    return []

def extract_list_key(val, key):
    """Extrai uma lista com o campo `key` de cada dict (ignorando vazios)."""
    lst = _to_list_of_dicts(val)
    return [d.get(key) for d in lst if isinstance(d, dict) and d.get(key)]

def normalize_str_list(L):
    """Limpa uma lista de strings: trim + remove vazios/None."""
    if not isinstance(L, list):
        return []
    return [x.strip() for x in L if isinstance(x, str) and x.strip()]

In [28]:
# Countries: códigos e nomes
kmm_opt["countries_codes"] = kmm_opt["production_countries"].map(lambda v: normalize_str_list(extract_list_key(v, "iso_3166_1")))
kmm_opt["countries_names"] = kmm_opt["production_countries"].map(lambda v: normalize_str_list(extract_list_key(v, "name")))

In [29]:
# Checagem
countries_flat = pd.Series(np.concatenate(kmm_opt["countries_names"].values)) if len(kmm_opt) else pd.Series([], dtype=object)
print("Qtd de países únicos:", countries_flat.nunique())
print(countries_flat.value_counts().head(10))

Qtd de países únicos: 160
United States of America    21130
United Kingdom               4089
France                       3932
Germany                      2248
Italy                        2166
Canada                       1763
Japan                        1645
Spain                         964
Russia                        912
India                         825
Name: count, dtype: int64


In [30]:
# Crio a lista de generos pelos nomes
kmm_opt["genres_list"] = kmm_opt["genres"].map(lambda v: normalize_str_list(extract_list_key(v, "name")))

In [31]:
# Checagem
genres_flat = pd.Series(np.concatenate(kmm_opt["genres_list"].values)) if len(kmm_opt) else pd.Series([], dtype=object)
print("Qtd de gêneros únicos:", genres_flat.nunique())
print(genres_flat.value_counts().head(20))

Qtd de gêneros únicos: 28
Drama              20231
Comedy             13171
Thriller            7611
Romance             6727
Action              6588
Horror              4666
Crime               4304
Documentary         3927
Adventure           3489
Science Fiction     3041
Family              2767
Mystery             2462
Fantasy             2308
Animation           1931
Foreign             1619
Music               1597
History             1397
War                 1322
Western             1042
TV Movie             764
Name: count, dtype: int64


In [32]:
kmm_opt = kmm_opt.drop(columns=["production_countries", "genres", "countries_codes"])

# K1 — 7.0 — SALVANDO A BASE DE DADOS OTIMIZADA

In [33]:
out_path = PATHS.PROC / "kaggle_otimizado.csv"
kmm_opt.to_csv(out_path, index=False)
print("Salvo em:", out_path.resolve())

Salvo em: /home/emersds/repos_projetos/project_lighthouse/data/processed/kaggle_otimizado.csv


# K1 — 8.0 — DECISÕES E PRÓXIMOS PASSOS
**Decisões nesta etapa**
- Mantive apenas as colunas úteis para negócio/modelagem (orçamento, receita, duração, votos/notas, popularidade, idioma, países, overview) e os campos de diretor/atores e ratings agregados.
  
- Padronizei year, normalizei título (original_title_norm) e garanti tipos numéricos coerentes (budget, revenue, runtime, vote_*, mean_rating, num_ratings).
  
- Deduplicate por (original_title_norm, year) mantendo o registro com maior vote_count.
  
- Salvei a versão de trabalho em data/processed/kaggle_otimizado.csv e preservei a versão completa com joins em data/intermediary/kmm_full.csv para auditoria.

**Saídas desta etapa**
- data/intermediary/kmm_full.csv
- data/processed/kaggle_otimizado.csv (saída principal)

**Próximos passos**
- Rodar EDA direcionada (K02).

- Testar hipóteses de negócio.